# **Topic Modeling avec LDA**

Auteurs : Tom LABIAUSSE - Pierre Ollivier - Amine CHERIF HAOUAT - Cyrine NABI

# 1# Imports



In [ ]:
# Imports des bibliothèques classiques
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import tqdm_notebook
tqdm_notebook().pandas()
from tqdm import tqdm

from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

# "lemmatizer" = FRENCH Tokenizer AND Lemmatizer (from spacy)
!python -m spacy download fr_core_news_md
import fr_core_news_md
lemmatizer = fr_core_news_md.load()

# Imports nécessaires pour LDA
import gensim
from gensim import models
from gensim.models import Phrases, CoherenceModel
from gensim import corpora

from nltk.tokenize import word_tokenize

# Imports pour l'optimisation Bayesienne
from bayes_opt import BayesianOptimization

In [ ]:
print(gensim.__version__)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
print("Connexion à Google Drive OK")

In [ ]:
path_to_GGDrive = "/content/gdrive/MyDrive/"

# 2# Chargement des données

### 2.1# Chargement des corpus

In [ ]:
def get_dataframe(file_name, column_texte, lower=False, to_spaces=[]):
    """ Charge la base de données dans une dataframe python en effectuant un pré-traitement de base :
        - suppression des lignes vides
        - suppression des sauts de lignes et indentations dans les resumes 
        to_spaces permet de préciser quels autres séquences doivent être remplacées par des espaces dans le texte."""
    input_df = pd.read_excel(file_name)
    df = input_df[['EAN',"Titre",column_texte]]
    df.columns = ['EAN','Titre','Texte']
    # Suppression des lignes vides
    df = df.dropna().reset_index(drop=True)
    # Suppression des sauts de lignes et indentations dans les resumes
    for k in range(0,df.shape[0]):
        for elt in to_spaces:
            df.iloc[k,1] = df.iloc[k,1].replace(elt," ")
        if lower:
            df.iloc[k,1] = df.iloc[k,1].lower()
    print('Dataframe with shape {1} loaded from "{0}"\n'.format(file_name,df.shape))
    return(df)

Si vous travaillez sur Colab, n'oubliez pas d'importer les fichiers corpus 1, corpus 2, stop_words_french et stop_symbols en cliquant sur la petite icône dossier à gauche.

Sinon, il faudra changer les chemin d'accès aux documents dans ce notebook.

In [ ]:
corpus1 = get_dataframe(path_to_GGDrive+'data_corpus/corpus 1.xlsx', column_texte = 'Description sans html', lower=True, to_spaces=["\n","\t","&nbsp;"])
corpus2 = get_dataframe(path_to_GGDrive+'data_corpus/corpus 2.xlsx', column_texte = 'Description', lower=True, to_spaces=["\n","\t","&nbsp;"])
corpus3 = get_dataframe(path_to_GGDrive+'data_corpus/corpus_editis.xlsx', column_texte = 'resume', lower=True, to_spaces=["\n","\t","&nbsp;"])

newcorpus = corpus1
corpus1and2 = newcorpus.append(corpus2, ignore_index=True)

### 2.2# Choix du corpus

In [ ]:
# Choisir entre corpus1, corpus2, corpus1and2 et corpus3
data = corpus1and2

# 3# Chargement des stopwords

In [ ]:
# stopwords spéciaux propres au corpus
special_stopwords = set(['',"",' '," ",'  ',"  ",'nbsp','pa','faire','al','grâce','homme','monde','grand', '\n','ouvrage','livre','nouveau','politique','pouvoir','histoire'])

# Mots récurrents pouvant être ajoutés au choix dans les stopwords ci-dessus :
# 'politique','pouvoir','histoire'

In [ ]:
# Chargement des stopwords francais & stopsymbols
f = open(path_to_GGDrive+'data_corpus/stop_words_french.txt') ; fr_stopwords = set(f.read().split('\n')) ; f.close() ; print("NB french stopwords : ",len(fr_stopwords))
f = open(path_to_GGDrive+'data_corpus/stop_symbols.txt') ; stopsymbols = set(f.read().split('\n')) ; f.close() ; print("NB stopsymbols : ",len(stopsymbols))
print("Nombre de 'special' stopwords : ",len(special_stopwords))
my_stopwords = fr_stopwords.union(stopsymbols,special_stopwords)
print("Nombre total de stopwords : ",len(my_stopwords))

In [ ]:
def hist_length(elements):
    stop_by_length = {}
    for e in elements:
        if len(e) in stop_by_length:
            stop_by_length[len(e)].append(e)
        else:
            stop_by_length[len(e)] = [e]
    sorted_tuples = sorted([(t,len(u)) for (t,u) in list(stop_by_length.items())], key = lambda x : x[0])
    plt.plot([x[0] for x in sorted_tuples],[x[1] for x in sorted_tuples])
    plt.title('Histogramme des tailles des éléments de "my_stopwords"')
    plt.xlabel("Taille") ; plt.ylabel("Effectif")
    return(stop_by_length)
stopwords_dico = hist_length(my_stopwords)

On n'observe pas de pic très étroit autour d'une taille typique. Il peut donc être intéréssant d'ordonner les stopwords par taille pour pouvoir les retrouver plus rapidement. On range les stopwords par taille dans le dictionnaire "stopwords_dico" dont les clefs sont les tailles.

# 4# Tokenization & Lemmatization

In [ ]:
# Exemple d'utilisation de "lemmatizer" A DECOMMENTER POUR FAIRE UN TEST
"""
txt = "Mangerai"
tokens_lemma = lemmatizer(txt) # dirons ! dirait
for token in tokens_lemma:
    print([token], "->", [token.lemma_])
"""

In [ ]:
# Définition une borne inf acceptable pour la taille des mots
taille_inf = 2

# Creation de la colonne des tokens lemmatisés
data['tokens'] = None

# Lemmatizer les resumes du corpus
len_stopwords = list(stopwords_dico.keys())
for k in tqdm(range(0,data.shape[0]), position=0, leave=True):
    txt = data['Titre'][k] + ' ' + data['Texte'][k]
    txt_tokens = []

    tokens_lemma = lemmatizer(str(txt))
    for token in tokens_lemma:
        lemma_token = str(token.lemma_).strip(" ") # Permet d'enlever les espaces superflus
        u = len(lemma_token)
        if (u >= taille_inf) and (not(u in len_stopwords) or not(lemma_token in stopwords_dico[u])):
            txt_tokens.append(lemma_token)
    data['tokens'][k] = txt_tokens

# 5# LDA

### 5.1# Data preparation

In [ ]:
tokens = data['tokens'].tolist()

> Préparation des modèles bi-grams et tri-grams

In [ ]:
## Décommenter la ligne bigram_model et la ligne tokens pour créer des groupes de 2 mots max
## Décommenter la ligne bigram_model, trigram_model et la ligne tokens pour créer des groupes de 3 mots max

# bigram_model = Phrases(tokens)
# trigram_model = Phrases(bigram_model[tokens], min_count=1)
# tokens = list(trigram_model[bigram_model[tokens]])

> Préparation des objets utiles à LDA

In [ ]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

### 5.2.1# Choix 1 : **Exécution** de LDA

In [ ]:
# Choix des hyperparamètres

num_topics = 30
alpha = 0.59
eta = 0.59
passes = 5

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                               id2word=dictionary_LDA,
                               num_topics=num_topics,
                               chunksize=100,
                               passes=passes,
                               alpha=alpha,
                               eta=eta,
                               random_state = 42)

coherence_model_lda = CoherenceModel(model=lda_model, texts=tokens, dictionary=dictionary_LDA, coherence='c_v')
print('Coherence Score: ',coherence_model_lda.get_coherence())

Pour sauvegarder le modèle :

In [ ]:
lda_model.save('lda_model.model')

### 5.2.2# Choix 2 : **Chargement d'un modèle** de LDA

In [ ]:
lda_model = gensim.models.LdaModel.load('lda_model.model')

### 5.3# Aperçu des topics

In [ ]:
# Aperçu pondéré des topics

for i,topic in lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=True):
    print(str(i)+": "+ topic)
    print()

In [ ]:
# Aperçu non pondéré des topics

lda_topics = lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=False)
for tp in lda_topics:
    tp_words = [wd[0] for wd in tp[1]]
    print("TOPIC {0} :".format(tp[0])," ".join(tp_words))

### 5.4# Allocation de topics aux documents et création d'un fichier csv contenant les résultats

In [ ]:
# Ajouter à la dataframe le topic de chaque document

data['Topic'] = None

for k in tqdm(range(data.shape[0]), position=0, leave=True):

  probas = []
  indexes = []
  prediction_k = lda_model[corpus[k]].copy()
  length = len(prediction_k)

  if length == 0:
    data['Topic'][k] = 'Sans topic'
  
  else:
    for j in range(length):
      indexes.append(prediction_k[j][0])
      probas.append(prediction_k[j][1])
    data['Topic'][k] = indexes[np.argmax(probas)]

Pour créer un fichier .csv contenant le topic de chaque document, exécuter la cellule suivante. Si vous travaillez sur coloab, le fichier se trouvera dans le même dossier que celui où vous avez importé les documents.

In [ ]:
# Pour créer un fichier csv contenant le topic de chaque document, exécuter cette cellule
pd.DataFrame(data[['EAN','Titre','Topic']]).to_csv('document_topic_allocation.csv', sep=';', index=False)

### 5.5# Prédiction de topics d'un nouveau document

In [ ]:
# Insérer l'exemple ici. Attention aux apostrophes et aux guillements!
document = "Cyril Lignac cuisine chez lui pour toi ! Envie d'une cuisine maison savoureuse et rapide ? En direct de sa cuisine, Cyril Lignac te propose 45 recettes salées et/ou sucrées pour mettre un peu de peps dans ton quotidien. Un risotto aux coquillettes, un poisson au four à l'huile vierge et aux petits légumes ou encore une fabuleuse tarte aux fraises ou des petits pots de crème à la vanille... Tu vas te régaler en toute simplicité ! Un livre indispensable, ultra-pratique et sans prétention, pour égayer tes déjeuners et dîners ; des recettes gourmandes, croquantes, craquantes, à déguster en solo, à deux, en famille ou entre amis. Avec Cyril, le fait-maison c'est ultra-facile ! Mets ton tablier et laisse-toi guider par ses précieux conseils et ses recettes ultra-réconfortantes."

In [ ]:
token_exeample = word_tokenize(document)
topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(token_exeample)]], columns=['topic #', 'weight', 'words in topic'])

# 6# Optimisation des paramètres



### 6.1# Préparation des outils d'optimisation

In [ ]:
def print_inventory(dct):
    print("Items held:")
    for item, amount in dct.items():  # dct.iteritems() in Python 2
        print("{} ({})".format(item, amount))

In [ ]:
# Fonction support
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                               id2word=dictionary_LDA,
                               num_topics=num_topics,
                               chunksize=100,
                               passes=5,
                               alpha=a,
                               eta=b,
                               random_state = 42)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokens, dictionary=dictionary_LDA, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

### 6.2# Grid search

In [ ]:
grid = {}
grid['Validation_Set'] = {}

# Range du nombre de topics
min_topics = 4
max_topics = 7
step_size = 1
topics_range = range(min_topics, max_topics+1, step_size)

# Paramètre alpha
alpha = list(np.arange(0.1, 1.01, 0.1))

# Paramètre bêta
beta = list(np.arange(0.1, 1.01, 0.1))

model_results_grid = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }


In [ ]:
# Prend beaucoup de temps à tourner

pbar = tqdm(total=(len(beta)*len(alpha)*len(topics_range)))
    
# iterate through number of topics
for k in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
        for b in beta:
            # get the coherence score for the given parameters
            cv = compute_coherence_values(corpus=corpus, dictionary=dictionary_LDA, 
                                                  k=k, a=a, b=b)
            # Save the model results
            model_results_grid['Topics'].append(k)
            model_results_grid['Alpha'].append(a)
            model_results_grid['Beta'].append(b)
            model_results_grid['Coherence'].append(cv)
                    
            pbar.update(1)
pd.DataFrame(model_results_grid).to_csv('lda_tuning_results_grid_search.csv', sep=',', index=False)
pbar.close()

### 6.3# Random search

In [ ]:
model_results_random = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [ ]:
# Nombre d'itérations
iter = 10

# limite inférieure du nombre de topics
nb_topic_inf = 7

# limite supérieure du nombre de topics
nb_topic_sup = 11

# Can take a long time to run

for j in tqdm(range(iter), position=0, leave=True):
  nb_topics = np.random.randint(nb_topic_inf,nb_topic_sup)
  alpha = np.random.random()
  beta = np.random.random()

  cv = compute_coherence_values(corpus=corpus, dictionary=dictionary_LDA, 
                                                  k=nb_topics, a=alpha, b=beta)
  # Save the model results
  model_results_random['Topics'].append(nb_topics)
  model_results_random['Alpha'].append(alpha)
  model_results_random['Beta'].append(beta)
  model_results_random['Coherence'].append(cv)

pd.DataFrame(model_results_random).to_csv('lda_tuning_results_random.csv', index=False)

### 6.4# Optimisation bayésienne

In [ ]:
# Choix de la région de l'espace des paramètres
pbounds = {'a': (0.01, 0.7), 'b': (0.01, 0.7)}

In [ ]:
import tqdm
pbar = tqdm.tqdm(10)

liste=[]
max=0
a=0
b=0
Kopt=0

for k in range(6,13): #on fait une optimisation bayésienne pour chaque nombre de topic k dans une plage choisie
  
  def black_box_function(a, b): 
    return compute_coherence_values(corpus, dictionary_LDA, k, a, b)

  optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2, 
    random_state=1,
  )
 
  optimizer.maximize(
      init_points=0,  # choix du nombre d'initialisation lors d'une optimisation
      n_iter=5,  # choix du nombre d'itération pour chaque optimisation 
  )
  liste.append(k)
  liste.append(optimizer.max)
  if max< optimizer.max['target']:
    max=optimizer.max['target']
    parametres=optimizer.max['params']
    a=parametres['a']
    b=parametres['b']
    Kopt=k
  pbar.update(1)
pbar.close()
print(liste)


print("Le nombre de topic optimal est ", Kopt)
print("Alpha optimal est  ", a)
print("Beta optimal est  ", b)
print("Pour un score de cohérence de ")
print(optimizer.max['target'])

# 7# Exploration avancée des topics de LDA

#### Allocation des topics dans tous les documents

In [ ]:
topics = [lda_model[corpus[i]] for i in range(len(data))]

In [ ]:
# Définition de la fonction d'allocation des poids de chaque topic à un document donné

def topics_document_to_dataframe(topics_document, num_topics):
    res = pd.DataFrame(columns=range(num_topics))
    for topic_weight in topics_document:
        res.loc[0, topic_weight[0]] = topic_weight[1]
    return res

In [ ]:
# Les lignes correspondent aux documents, les colonnes aux topics, et la case (i,j) correspondant au poids du topic j dans le document i
document_topic = \
pd.concat([topics_document_to_dataframe(topics_document, num_topics=num_topics) for topics_document in topics]) \
  .reset_index(drop=True).fillna(0)

In [ ]:
document_topic.head()

In [ ]:
# Exemple : affichage par ordre décroissant de poids des documents associés au topic 5
topic = 0

document_topic.sort_values(topic, ascending=False)[topic].head(10)

#### Observation de la distribution des topics dans tous les documents

In [ ]:
%matplotlib inline
import seaborn as sns; sns.set(rc={'figure.figsize':(30,50)})
sns.heatmap(document_topic.loc[document_topic.idxmax(axis=1).sort_values().index])

In [ ]:
# Nombre de documents appartenant à chaque topic

sns.set(rc={'figure.figsize':(10,5)})
document_topic.idxmax(axis=1).value_counts().plot.bar(color='lightblue')